In [40]:
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import preprocess_documents
import json

word2vec_model = Word2Vec.load('word2vec.model')

In [41]:
jobs_dataset = json.load(open('./data/final/jobs.json'))
jobs_dataset = preprocess_documents(jobs_dataset)

for i in range(len(jobs_dataset)):
    for j in range(len(jobs_dataset[i])):
        jobs_dataset[i][j] = word2vec_model.wv[jobs_dataset[i][j]]
jobs_dataset[0]

[array([-0.7304232 ,  0.19612044,  0.92002285, -0.11172968,  0.98880327,
        -0.32300565, -0.7649982 , -0.21409005, -2.1803973 , -0.2330147 ,
        -1.9736768 ,  0.2697791 ,  0.62939113, -0.01977196,  0.625636  ,
        -0.54969037,  1.0371788 ,  0.04494958,  1.7254237 , -0.53548807,
        -0.7404606 ,  0.02589464,  1.2660213 ,  2.4440613 ,  0.33295298,
        -0.7645274 , -1.0453076 , -0.3095146 , -0.48243436,  0.82412845,
         0.8551301 , -1.3547728 , -0.45349917, -1.3915691 ,  0.8152827 ,
         0.11548156,  0.32609493, -1.0599602 ,  0.42152476,  0.85823005,
         0.76996773, -0.754073  , -0.3060641 ,  1.1161501 ,  0.5619146 ,
         0.05145977,  0.37013903, -1.1151251 , -0.75381196,  1.4578123 ,
        -1.3680246 ,  1.432716  ,  0.21504265,  0.82030267,  0.11612281,
        -1.4084257 ,  1.1109533 ,  0.90773267,  1.1052195 , -2.1591775 ,
         1.4041731 , -1.6621208 , -1.66014   , -0.6792017 ,  1.1299571 ,
         1.4033724 , -0.40112707,  0.5723509 ,  1.3

In [172]:
import random
def random_chunk():
    file = random.choice(jobs_dataset)
    while len(file) < 5:
        file = random.choice(jobs_dataset)
    start = 0
    end = len(file)-1
    return file[start:end]

def random_training_set():    
    chunk = random_chunk()
    inp = Variable(torch.Tensor(chunk[:-1]))
    target = Variable(torch.Tensor(chunk[1:]))
    return inp, target

In [173]:
import torch
import torch.nn as nn
from torch.autograd import Variable


class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = input.view(1, -1).cuda()
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size)).cuda()

In [174]:
decoder  = RNN(100, 100, 100, 2).cuda()

In [175]:
def train(inp, target):
    hidden = decoder.init_hidden().cuda()
    decoder.zero_grad()
    loss = 0
    
    for c in range(len(inp)):
        output, hidden = decoder(inp[c].cuda(), hidden)
        loss += criterion(output, target[c].view(1, -1).cuda())

    loss.backward()
    decoder_optimizer.step()

    return loss.data.item() / len(inp)


In [ ]:
n_epochs = 12000
print_every = 10
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.015

decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.MSELoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in tqdm(range(1, n_epochs + 1)):
    loss = train(*random_training_set())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 50, loss))
        print(evaluate())
#         print(evaluate('ge', 200), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0








  0%|          | 0/12000 [00:00<?, ?it/s]






  0%|          | 1/12000 [00:00<25:30,  7.84it/s]






  0%|          | 2/12000 [00:00<23:52,  8.38it/s]






  0%|          | 4/12000 [00:00<21:07,  9.47it/s]






  0%|          | 5/12000 [00:00<22:11,  9.01it/s]






  0%|          | 6/12000 [00:00<22:22,  8.93it/s]






  0%|          | 7/12000 [00:00<24:32,  8.14it/s]






  0%|          | 8/12000 [00:00<23:52,  8.37it/s]






  0%|          | 10/12000 [00:01<20:58,  9.53it/s]






  0%|          | 12/12000 [00:01<18:02, 11.07it/s]

 1s (10 0%) 0.4857]
want 
superior 
futur 
increas 
monei 
earn 
futur 
increas 
monei 
earn 
hello want superior futur increas monei earn futur increas monei earn 









  0%|          | 14/12000 [00:01<20:29,  9.75it/s]






  0%|          | 16/12000 [00:01<21:40,  9.21it/s]






  0%|          | 19/12000 [00:01<20:39,  9.67it/s]






  0%|          | 21/12000 [00:02<24:20,  8.20it/s]

 2s (20 0%) 1.6642]
want 
superior 
bud 
gentleman 
bud 
hei 
bud 
increas 
bud 
increas 
hello want superior bud gentleman bud hei bud increas bud increas 









  0%|          | 22/12000 [00:02<25:43,  7.76it/s]






  0%|          | 23/12000 [00:02<24:05,  8.29it/s]






  0%|          | 25/12000 [00:02<20:57,  9.53it/s]






  0%|          | 27/12000 [00:02<22:43,  8.78it/s]






  0%|          | 29/12000 [00:03<19:59,  9.98it/s]






  0%|          | 31/12000 [00:03<20:21,  9.80it/s]

...








 54%|█████▍    | 6483/12000 [08:31<08:01, 11.46it/s]






 54%|█████▍    | 6485/12000 [08:31<07:11, 12.77it/s]






 54%|█████▍    | 6487/12000 [08:31<07:55, 11.59it/s]






 54%|█████▍    | 6489/12000 [08:31<08:10, 11.24it/s]






 54%|█████▍    | 6491/12000 [08:31<08:34, 10.70it/s]

 31s (6490 27%) 0.4669]
want 
pleasant 
pleasant 
pleasant 
futur 
futur 
prais 
special 
assist 
assist 
hello want pleasant pleasant pleasant futur futur prais special assist assist 









 54%|█████▍    | 6493/12000 [08:31<08:46, 10.47it/s]






 54%|█████▍    | 6495/12000 [08:32<07:47, 11.78it/s]






 54%|█████▍    | 6497/12000 [08:32<07:14, 12.68it/s]






 54%|█████▍    | 6499/12000 [08:32<07:35, 12.08it/s]






 54%|█████▍    | 6501/12000 [08:32<06:53, 13.30it/s]

 32s (6500 27%) 1.9594]
want 
want 
want 
futur 
increas 
monei 
monei 
special 
todai 
assist 
hello want want want futur increas monei monei special todai assist 









 54%|█████▍    | 6503/12000 [08:32<08:43, 10.50it/s]






 54%|█████▍    | 6505/12000 [08:32<08:29, 10.78it/s]






 54%|█████▍    | 6507/12000 [08:33<09:46,  9.36it/s]

In [178]:
def evaluate(prime_str='hello', predict_len=10, temperature=0.8):
    hidden = decoder.init_hidden()
    
    prime_input = torch.Tensor([word2vec_model.wv[prime_str]]).cuda()
    predicted = prime_str + ' '

    # Use priming string to "build up" hidden state
    for p in range(len(prime_input) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        
        # Add predicted character to string and use as next input
        predicted_char = word2vec_model.wv.similar_by_vector(output.view(-1).cpu().detach().numpy())[0][0] + ' '
        print(predicted_char)
        predicted += predicted_char
        inp = output

    return predicted






 14%|█▎        | 1627/12000 [02:34<15:09, 11.40it/s]

In [ ]:
evaluate()